# A.1 Install Required Packages

In [ ]:
# Install packages
!pip install ultralytics
!pip install google-cloud-vision

# A.2 Import Required Libraries

In [ ]:
# Import essential libraries
import os
from ultralytics import YOLO
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from google.colab import drive, files
import re
from google.cloud import vision

# A.3 Open File Dialog to Select an Image

In [ ]:
# Function to open file dialog and select an image
# Upload the image
uploaded = files.upload()
# Extract the filename
filename = list(uploaded.keys())[0]
# Read the image using cv2
image = cv2.imread(filename)
# Make a copy to display it later
original_image = image.copy()
# Convert the image to RGB (for displaying with matplotlib)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# Save the image
plt.savefig('Original Image.jpg')
# Display the image
plt.imshow(image_rgb)
plt.axis('off')
plt.show()

# A.4 Vehicle Detection

In [ ]:
# Load YOLO11n model pre-trained on COCO dataset
vehicle_detector = YOLO("yolov11n.pt")  # Load pre-trained YOLO11n model

# COCO class IDs for vehicles: car, bus, truck
vehicle_classes = [2, 5, 7]

# Run inference on the input image
vehicles = vehicle_detector(image)[0]

# List to store detection data [x1, y1, x2, y2, confidence] for vehicles
vehicle_detections = []
# List to store bounding boxes [x1, y1, x2, y2] for cropping
vehicle_bboxes = []

# Process each detected object to filter vehicles
for detection in vehicles.boxes.data.tolist():
  x1, y1, x2, y2, score, class_id = detection
  if int(class_id) in vehicle_classes and score > 0.70:
    vehicle_detections.append([x1, y1, x2, y2, score])

    if score < 0.7:  # Skip low-confidence detections
      continue

    # Crop vehicle from the original image
    cropped_vehicle = original_image[int(y1):int(y2), int(x1): int(x2), :]
    vehicle_bboxes.append([x1, y1, x2, y2])

    # Draw bounding box on the output image
    cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 15)
    # Print the result
    print(f'{vehicle_detector.names[class_id]} {score:.2f}')
    print(f'coordinates are (({x1}, {y1}), ({x2}, {y2}))')

# Convert the images to RGB (for displaying with matplotlib)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
cropped_vehicle_rgb = cv2.cvtColor(cropped_vehicle, cv2.COLOR_BGR2RGB)

# Save images to pass them to the OCR
try:
  Image.fromarray(lp_rgb).save('Vehicle Detection - raw.jpg')
  Image.fromarray(lp_rgb).save('Cropped vehicle.jpg')
  print(f'Saved: {save_path}')
except Exception as e:
  print(f'Error while saving {save_path}: {e}')

# Display the image
fig = plt.figure(figsize=(10, 7))
# Add the Original image to the figure
plt.subplot(1, 2, 1)  # 1 rows, 2 columns, first position
plt.imshow(image_rgb)
plt.axis('off')  # Hide the axis labels
plt.title("Original image")
# Add the Vehicle image to the figure
plt.subplot(1, 2, 2)  # 1 rows, 2 columns, second position
plt.imshow(cropped_vehicle_rgb)
plt.axis('off')  # Hide the axis labels
plt.title("Vehicle")
# Set the figure title
subtitle = fig.suptitle('Vehicle Detection')
subtitle.set(**{'fontsize': 24})
# Save the image
plt.savefig('Vehicle Detection.jpg')
# Show the image
plt.show()

# A.5 License Plate Detection

In [ ]:
# Load custom-trained YOLO11n model for license plate detection
license_plate_detector = YOLO('license_plate_detector.pt')

# Detect license plates in the cropped vehicle image
license_plates = license_plate_detector(cropped_vehicle)[0]

x1_vehicle, y1_vehicle, x2_vehicle, y2_vehicle = vehicle # position of cropped vehicle
# List to store all license plate data
lp = []  # Format: [adjusted_x1, adjusted_y1, adjusted_x1, adjusted_y1]

# Process each detected license plate
for license_plate in license_plates.boxes.data.tolist():
  x1, y1, x2, y2, score, class_id = license_plate

  if score < 0.7:  # Skip low-confidence detections
    continue

  # License plate cropping
  cropped_license_plate = cropped_vehicle[int(y1):int(y2), int(x1): int(x2), :]

  # Adjust coordinates to original image
  adjusted_x1 = x1_vehicle + x1
  adjusted_y1 = y1_vehicle + y1
  adjusted_x2 = x1_vehicle + x2
  adjusted_y2 = y1_vehicle + y2

  # Store LP data
  lp = [adjusted_x1, adjusted_y1, adjusted_x1, adjusted_y1]

  # Draw bounding box on the original image
  cv2.rectangle(image, (int(adjusted_x1), int(adjusted_y1)), (int(adjusted_x2), int(adjusted_y2)), (0, 0, 255), 15)
  # Print the result
  print(f'{license_plate_detector.names[class_id]} {score:.2f}')
  print(f'coordinates are (({adjusted_x1}, {adjusted_y1}), ({adjusted_x2}, {adjusted_y2}))')


# Convert the image to RGB (for displaying with matplotlib)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
cropped_lp_rgb = cv2.cvtColor(cropped_license_plate, cv2.COLOR_BGR2RGB)

# Save images to pass them to the OCR
try:
  Image.fromarray(image_rgb).save('License Plate Detection - raw.jpg')
  Image.fromarray(cropped_lp_rgb).save('Cropped license plate.jpg')
  print(f'Saved: {save_path}')
except Exception as e:
  print(f'Error while saving {save_path}: {e}')

# Display the image
fig = plt.figure(figsize=(10, 7))
# Add the Original image to the figure
plt.subplot(1, 2, 1)  # 1 rows, 2 columns, first position
plt.imshow(image_rgb)
plt.axis('off')  # Hide the axis labels
plt.title("Original image")
# Add the Original image to the figure
plt.subplot(1, 2, 2)  # 1 rows, 2 columns, second position
plt.imshow(cropped_lp_rgb)
plt.axis('off')  # Hide the axis labels
plt.title("License Plate")
# Set the figure title
subtitle = fig.suptitle('License Plate Detection')
subtitle.set(**{'fontsize': 24})
# Save the image
plt.savefig('License Plate Detection.jpg')
plt.show()

# A.6 License Plate Segmentation

In [ ]:
# Load custom-trained YOLO11n model for license plate segmentation
license_plate_segmenter = YOLO('license_plate_segmenter.pt')

# Detect license plate's key components in the cropped license plate image
results = license_plate_segmenter(cropped_license_plate)[0]

x1_lp, y1_lp, x2_lp, y2_lp = lp # position of cropped license plate

# Process each detected license plate's components
for result in results.boxes.data.tolist():
  x1, y1, x2, y2, score, class_id = result

  # State code
  if int(class_id) == 0:
    # State code cropping
    cropped_state_code = cropped_license_plate[int(y1):int(y2), int(x1): int(x2), :]
    # Set image path
    save_path = f'cropped_state_code.jpg'
    pil_img = Image.fromarray(cropped_state_code)

  # Numbers
  elif int(class_id) == 1:
    # Numbers cropping
    cropped_numbers = cropped_license_plate[int(y1):int(y2), int(x1): int(x2), :]
    # Set image path
    save_path = f'cropped_numbers.jpg'
    pil_img = Image.fromarray(cropped_numbers)

  # Save images to pass them to the OCR
  try:
    pil_img.save(save_path)
    print(f'Saved: {save_path}')
  except Exception as e:
    print(f'Error while saving {save_path}: {e}')

  # Adjust coordinates to original image
  adjusted_x1 = x1_lp + x1
  adjusted_y1 = y1_lp + y1
  adjusted_x2 = x1_lp + x2
  adjusted_y2 = y1_lp + y2

  # Draw bounding box on the original image
  cv2.rectangle(image, (int(adjusted_x1), int(adjusted_y1)), (int(adjusted_x2), int(adjusted_y2)), (255, 0, 0), 10)
  # Print the result
  print(f'{license_plate_splitor.names[class_id]} {score:.2f}')
  print(f'coordinates are (({adjusted_x1}, {adjusted_y1}), ({adjusted_x2}, {adjusted_y2}))')

# Convert the image to RGB (for displaying with matplotlib)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
cropped_state_code_rgb = cv2.cvtColor(cropped_state_code, cv2.COLOR_BGR2RGB)
cropped_numbers_rgb = cv2.cvtColor(cropped_numbers, cv2.COLOR_BGR2RGB)


# Save images to pass them to the OCR
save_path = f'License Plate Segmentation - raw.jpg'
try:
  Image.fromarray(image_rgb).save(save_path)
  print(f'Saved: {save_path}')
except Exception as e:
  print(f'Error while saving {save_path}: {e}')

# Display the image
fig = plt.figure(figsize=(10, 7))
# Add the Original image to the figure
plt.subplot(1, 3, 1)  # 1 rows, 3 columns, first position
plt.imshow(image_rgb)
plt.axis('off')  # Hide the axis labels
plt.title("Original image")
# Add the Original image to the figure
plt.subplot(1, 3, 2)  # 1 rows, 3 columns, second position
plt.imshow(cropped_state_code_rgb)
plt.axis('off')  # Hide the axis labels
plt.title("License Plate State Code")
# Add the Original image to the figure
plt.subplot(1, 3, 3)  # 1 rows, 3 columns, third position
plt.imshow(cropped_numbers_rgb)
plt.axis('off')  # Hide the axis labels
plt.title("License Plate Numbers")
# Set the figure title
plt.title('License Plate Detection')
# Set the figure title
subtitle = fig.suptitle('License Plate Segmentation')
subtitle.set(**{'fontsize': 24})
# Save the image
plt.savefig('License Plate Segmentation.jpg')
plt.show()

# A.7 Character Recognition

In [ ]:
# Set Google Cloud credentials for Vision API access
# Required for authenticating requests to Google's services
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] ='vision_key.json'
# Regular expression pattern to match word characters (for potential text cleaning)
WORD = re.compile(r"\w+")

# Function to detect text in an image using Google Cloud Vision API
def detect_text(path):
  # Initialize Vision API client
  client = vision.ImageAnnotatorClient()

  # Read image file as binary data
  with open(path, "rb") as image_file:
      content = image_file.read()

  # Create Vision API image object
  image = vision.Image(content=content)

  # Perform text detection (for sparse text)
  # for non-dense text
  response = client.text_detection(image=image)
  # for dense text
  texts = response.text_annotations
  ocr_text = []

  # Process detected text annotations
  for text in texts:
    ocr_text.append(text.description)

    # Capture bounding box coordinates
    vertices = [f"({vertex.x},{vertex.y})" for vertex in text.bounding_poly.vertices]

  # Handle API errors
  if response.error.message:
      raise Exception(
          "{}\nFor more info on error messages, check: "
          "https://cloud.google.com/apis/design/errors".format(response.error.message)
      )
  return vertices, ocr_text

# Process state code image (first part of license plate)
SC_vertices, SC_text = detect_text('cropped_state_code.jpg')

# Process license plate numbers image (second part of license plate)
N_vertices, N_text = detect_text('cropped_numbers.jpg')

# Display cleaned OCR results
print(SC_text[0]) # First element contains full detected text
print(N_text[0].replace(" ", "")) # Remove spaces from numbers output